In [29]:
from ZW_model import LSTM
from ZW_utils import std_classes,dataloading
from ZW_dataset import PSI_Dataset
import numpy as np

classes = std_classes
data_split_ratio = 0.85
batch_size = 100
max_epochs = 30
learning_rate = 0.1
model = LSTM(hidden_size=12,out_features=1)
augmentation = False
uniqueness = False

In [30]:

from config import DATA_DIRECTORY
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

# layouts = np.load(DATA_DIRECTORY/"v22DF_m2_sorted_layouts.npy", allow_pickle=True)
# results = np.load(DATA_DIRECTORY/"v22DF_m2_sorted_results.npy", allow_pickle=True)
layouts = np.load("GPT_NA/initial_10k.npy", allow_pickle=True)
results = np.load("GPT_NA/results_initial_10k.npy", allow_pickle=True)
l2 = []
r2 = []
cutoff = 143.957
for i,r in enumerate(results):
    if r > 0:
        l2.append(layouts[i])
        r2.append(r)
layouts = np.asanyarray(l2)
results = np.asanyarray(r2)
from split_functions import uniqueness_check
esigns,equipments = uniqueness_check(layouts)
sorted_equipments = equipments.copy()
sorted_equipments.sort()
sorted_results = []
for se in sorted_equipments:
    index = equipments.index(se)
    sorted_results.append(results[index])
eq_array = np.zeros((len(sorted_equipments),22))
for i,e in enumerate(sorted_equipments):
    for j,u in enumerate(e):
        eq_array[i,j] = u
re_array = np.array(sorted_results)
equipment_chunks = []
results_chunks = []
for equipment in sorted_equipments:
    for i in range(len(equipment)):
        candidate_chunk = equipment[:i+1]
        if candidate_chunk not in equipment_chunks:
            equipment_chunks.append(candidate_chunk)
            #checking the same chunks in eq array
            chunk_indices = np.where((eq_array[:,:i+1] == candidate_chunk).all(axis=1))[0]
            chunk_results = np.mean(re_array[chunk_indices])
            results_chunks.append(chunk_results)

In [32]:
#one hot encoding from equipment chunks
lengths = [len(e) for e in equipment_chunks]
max_length = max(lengths)
one_hot_chunks = []
for equipment in equipment_chunks:
    one_hot = np.zeros((max_length,12))
    for i,e in enumerate(equipment):
        one_hot[i,e] = 1
    one_hot_chunks.append(one_hot)
one_hot_chunks = np.array(one_hot_chunks)
results_chunks = np.array(results_chunks)
one_hot_tensors = torch.tensor(one_hot_chunks).float()
results_tensors = torch.tensor(results_chunks).float()
batch_size = 20
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
max_epochs = 10
train_loss = []
val_loss = []
for epoch in range(max_epochs):
    model.train()
    epoch_loss = 0
    for n in range(0, len(one_hot_tensors), batch_size):
        optimizer.zero_grad()
        batch_input = one_hot_tensors[n : n + batch_size]
        batch_output = results_tensors[n : n + batch_size]
        batch_lengths = lengths[n : n + batch_size]
        packed_batch_input = nn.utils.rnn.pack_padded_sequence(
            batch_input, batch_lengths, batch_first=True, enforce_sorted=False
        )
        y_pred = model(packed_batch_input)
        loss = torch.nn.functional.mse_loss(y_pred, batch_output.unsqueeze(1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss /= len(one_hot_tensors) / batch_size
    train_loss.append(epoch_loss)
    print(f"Epoch {epoch} loss: {epoch_loss}")

Epoch 0 loss: 28418.318143443412
Epoch 1 loss: 28413.81858330682
Epoch 2 loss: 28411.660144284262
Epoch 3 loss: 28411.753923678883
Epoch 4 loss: 28411.79763553312
Epoch 5 loss: 28411.8199966913
Epoch 6 loss: 28411.829346855746
Epoch 7 loss: 28411.835106254748
Epoch 8 loss: 28411.837021124546
Epoch 9 loss: 28411.836786089527


In [22]:
import torch.nn as nn
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer.zero_grad()

# y_pred = model(packed_x)
# loss = loss_function(y_pred, y)
# loss.backward()
# optimizer.step()
# epoch_loss += loss.item()
# _, predicted = torch.max(y_pred.data, 1)
# train_total += y.size(0)
# train_correct += (predicted == y).sum().item()
# epoch_loss = epoch_loss / len(train_loader)
# model.eval()
# correct = 0
# total = 0
# val_loss = 0

In [ ]:
data = PSI_Dataset(e, results, classes,block_size,training_type="standard")

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# training_set, validation_set = torch.utils.data.random_split(data, [int(data_split_ratio*len(data)), len(data)-int(data_split_ratio*len(data))])
# train_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(validation_set, batch_size=batch_size, shuffle=True)

# for epoch in range(max_epochs):
#     model.train()
#     for i, (x, y) in enumerate(train_loader):
#         optimizer.zero_grad()
#         y_pred = model(x)
#         y_pred = y_pred.view(y.size(0),block_size)
#         y = y.to(torch.float32)
#         loss = torch.nn.functional.mse_loss(y_pred, y)
#         loss.backward()
#         optimizer.step()
#     val_loss = 0
#     with torch.no_grad():
#         model.eval()
#         for i, (x, y) in enumerate(val_loader):
#             y_pred = model(x)
#             y_pred = y_pred.view(y.size(0),block_size)
#             y = y.to(torch.float32)
#             loss = torch.nn.functional.mse_loss(y_pred, y)
#             val_loss += loss.item()
#         val_loss /= len(val_loader)
#     print(f"Epoch {epoch} - Validation loss: {val_loss}")

In [7]:
model.eval()
model(torch.tensor([[0,1],[0,2],[0,3],[0,4],[0,5],[0,7],[0,9],[0,11]]))
# model(torch.tensor([[0,5,1],[0,5,2],[0,5,3],[0,5,4],[0,5,5],[0,5,7],[0,5,9],[0,5,11]]))
# model(torch.tensor([[0,5,4,1],[0,5,4,2],[0,5,4,3],[0,5,4,4],[0,5,4,5],[0,5,4,7],[0,5,4,9],[0,5,4,11]]))

tensor([[[255.2910],
         [213.3581]],

        [[255.2910],
         [255.8791]],

        [[255.2910],
         [211.1716]],

        [[255.2910],
         [255.9067]],

        [[255.2910],
         [255.8386]],

        [[255.2910],
         [255.8868]],

        [[255.2910],
         [210.0258]],

        [[255.2910],
         [248.1066]]], grad_fn=<ViewBackward0>)